# This is a nice visualization tool for the trigger turn-ons

In [1]:
import ROOT
ROOT.enableJSVis()
from ROOT import TFile, TCanvas, TGraph, TF1, TMath, TLine
from ROOT import gBenchmark, gStyle, gROOT
from ROOT import kBlack, kBlue, kRed, kCyan, kMagenta
from array import array
gStyle.SetOptStat(0)

Welcome to JupyROOT 6.14/00


## Here we import the latest data

In [31]:
dtfile = TFile("output-DATA-2a.root","READ")
mcfile = TFile("output-MC-2a.root","READ")
nufile = TFile("output-NU-2a.root","READ")
#file.cd("Standard/Eta_0.0-1.3/")
#file.ls()

In [32]:
Year = 16
if Year==16:
    triggers = ["0","40","60","80","140","200","260","320","400","450"]
else:
    triggers = ["0","40","60","80","140","200","260","320","400","450","500"]
hpts = []
hpt0s = []
mchpts = []
mchpt0s = []
mchpt = mcfile.Get("Standard/Eta_0.0-1.3/mc/hpt")
mchpt0 = mcfile.Get("Standard/Eta_0.0-1.3/mc/hpt0")
nuhpt = nufile.Get("Standard/Eta_0.0-1.3/mc/hpt")
nuhpt0 = nufile.Get("Standard/Eta_0.0-1.3/mc/hpt0")
for trg in triggers:
    hpts.append(dtfile.Get("Standard/Eta_0.0-1.3/jt"+trg+"/hpt"))
    hpt0s.append(dtfile.Get("Standard/Eta_0.0-1.3/jt"+trg+"/hpt0"))
    mchpts.append(mcfile.Get("Standard/Eta_0.0-1.3/jt"+trg+"/hpt"))
    mchpt0s.append(mcfile.Get("Standard/Eta_0.0-1.3/jt"+trg+"/hpt0"))

# Choose parameters and optionally, look at the data

In [71]:
Trg = 9
Preview = True
RunParams = []
if Year==16:
    if Trg==1: # HLTPFJet40
        RunParams = [0.95,1.05,15,90,-0.1,0.1,1,1.025]
    if Trg==2: # HLTPFJet60
        RunParams = [0.95,1.05,40,130,-0.1,0.1,1,1.0]
    if Trg==3: # HLTPFJet80
        RunParams = [0.95,1.05,62,154,-0.1,0.1,1,1.0]
    if Trg==4: # HLTPFJet140
        RunParams = [0.95,1.05,80,245,-0.1,0.1,1,0.99]
    if Trg==5: # HLTPFJet200
        RunParams = [0.95,1.05,145,300,-0.1,0.1,1,1]
    if Trg==6: # HLTPFJet260
        RunParams = [0.95,1.05,205,362,-0.1,0.1,1,0.99]
    if Trg==7: # HLTPFJet320
        RunParams = [0.95,1.05,266,430,-0.1,0.1,1,1]
    if Trg==8: # HLTPFJet400
        RunParams = [0.95,1.05,330,548,-0.1,0.1,1,1]
    if Trg==9: # HLTPFJet450
        RunParams = [0.95,1.05,408,686,0.1,0.22,1,1]
ymin = RunParams[0]
ymax = RunParams[1]
xmin = RunParams[2]
xmax = RunParams[3]
cmin = RunParams[4]
cmax = RunParams[5]
shift = RunParams[6]
scale = RunParams[7]

Turn = hpts[Trg].Clone("hpt_"+triggers[Trg])
Turn.Divide(hpts[Trg-shift])
Turn.Scale(1.0/scale)
Turn_0 = hpt0s[Trg].Clone("hpt0_"+triggers[Trg])
Turn_0.Divide(hpt0s[Trg-shift])
Turn_0.Scale(1.0/scale)

MCTurn = mchpts[Trg].Clone("hpt_"+triggers[Trg])
MCTurn.Divide(mchpts[Trg-shift])
MCTurn_0 = mchpt0s[Trg].Clone("hpt0_"+triggers[Trg])
MCTurn_0.Divide(mchpt0s[Trg-shift])
MCTurn.SetLineColor(kRed)
MCTurn_0.SetLineColor(kRed)

if Preview:
    c = TCanvas("c","c",900,800)
    Turn_0.SetAxisRange(300,xmax,"X")
    Turn_0.SetAxisRange(0,1.2,"Y")
    Turn_0.Draw()
    Turn.Draw("SAME")
    MCTurn_0.Draw("SAME")
    MCTurn.Draw("SAME")
    c.Draw()
    c.SetLogx()
    c.Update()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


## Fit type "square fraction": $\frac{p_0}{2}\left(\frac{x-p_2}{\sqrt{p_1^2+(x-p_2)^2}}+1\right) + p_3$

In [73]:
def Fun3(x,par):
    return par[0]*((x[0]-par[2])/pow(pow(par[1],2)+pow((x[0]-par[2]),2),0.5)+1)/2.0+par[3]
#def Fun3(x,par):
#    return par[0]/(1+TMath.Exp(-par[1]*(x[0]-par[2])))+par[3]

fit3 = TF1("squarefrac",Fun3,xmin,xmax,4)
fit3.SetParLimits(0,ymin-cmin,ymax-cmin);
fit3.SetParLimits(1,0.0001,1000);
fit3.SetParLimits(2,xmin,xmax);
fit3.SetParLimits(3,cmin,cmax);
fit3.SetParameters((ymin+ymax)/2.0,20,(xmin+xmax)/2.0,(cmin+cmax)/2.0);

c3 = TCanvas("c3","c3",900,800)
Turn_0.Fit(fit3,"M","SAME",xmin,xmax)
Turn_0.SetAxisRange(300,xmax,"X")
Turn_0.SetAxisRange(0,1.2,"Y")
Turn_0.Draw()
Turn.Draw("SAME")
MCTurn_0.Draw("SAME")
MCTurn.Draw("SAME")
c3.Draw()
c3.SetLogx()

p3s = [fit3.GetParameter(0),fit3.GetParameter(1),fit3.GetParameter(2),fit3.GetParameter(3)]
turnpoint95 = p3s[2]+p3s[1]/TMath.Sqrt((TMath.Power(0.9-0.1*p3s[3]/p3s[0],-2)-1))
turnval95 = fit3.Eval(turnpoint95)
turnpoint99 = p3s[2]+p3s[1]/TMath.Sqrt((TMath.Power(0.98-0.02*p3s[3]/p3s[0],-2)-1))
turnval99 = fit3.Eval(turnpoint99)
maxval = p3s[0]+p3s[3]
print(turnpoint95,turnval95,turnval95/maxval)
print(turnpoint99,turnval99,turnval99/maxval)
l95 = TLine(turnpoint95,0,turnpoint95,1)
l95.SetLineColor(kMagenta)
l95.Draw()
l99 = TLine(turnpoint99,0,turnpoint99,1)
l99.SetLineColor(kCyan)
l99.Draw()
l1 = TLine(xmin,1,xmax,1)
l1.SetLineColor(kBlack)
l1.Draw()

c3.Update()

508.51516276908325 0.9566492950338851 0.9499999999999998
580.481878526521 0.9969292653511013 0.9899999999999999
 FCN=1436.27 FROM HESSE     STATUS=OK             23 CALLS         217 TOTAL
                     EDM=7.12088e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           8.74582e-01   1.57952e-03   2.90454e-04   9.95767e-03
   2  p1           2.68593e+01   1.36669e-01   5.82249e-06   9.39260e-01
   3  p2           4.57498e+02   6.20807e-02   6.20193e-06  -8.96999e-01
   4  p3           1.32417e-01   8.33385e-04   8.31211e-05   5.44951e-02


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 


## Notes

We have fitted a function to the fraction of hpt0 (leading jet inclusive histogram). The fraction is provided by comparing adjacent triggers (e.g. PFJet60 to PFJet40). The histogram hpt (inclusive histogram for all jets) is more stable, but more scarce, and hence hpt0 is a better fitting target. Notably, hpt0 tends to  overshoot slightly, but the fits are still quite good.

Sometimes the histogram hpt does not stabilize at 1.0 on high pT values, and hence we sometimes scale the fraction histograms slightly. Two points from the fit are given: 95% of the maximum value and 99% of the maximum value. A classical turn-on point with these fits is generally reached at the 95% point.

In [63]:
cu = TCanvas("cu","cu",900,800)
mchpt0.Draw()
mchpt0s[1].Draw("SAME")
mchpt0s[2].Draw("SAME")
mchpt0s[3].Draw("SAME")
mchpt0s[4].Draw("SAME")
cu.Draw()
cu.SetLogx()
cu.SetLogy()
cu.Update()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cu


In [74]:
Trgf = 1
mcscale = 1.244e8
nuscale = 3.13e6
donu = True
dof  = True

cuf = TCanvas("cuf","cuf",900,800)
if dof and Trgf>0:
    FTurn = hpts[Trgf].Clone("ffhpt_"+triggers[Trgf])
    FTurn.Divide(hpts[Trgf-1])
    FTurn_0 = hpt0s[Trgf].Clone("ffhpt0_"+triggers[Trgf])
    FTurn_0.Divide(hpt0s[Trgf-1])
    FTurn.SetLineColor(kMagenta)
    FTurn_0.SetLineColor(kMagenta)
    FTurn.Draw("SAME")
    FTurn_0.Draw("SAME")
Frac = hpts[Trgf].Clone("fhpt_"+triggers[Trgf])
Frac_0 = hpt0s[Trgf].Clone("fhpt0_"+triggers[Trgf])
Frac.Divide(mchpt)
Frac_0.Divide(mchpt0)
Frac.Scale(1.0/mcscale)
Frac_0.Scale(1.0/mcscale)
Frac.Draw("SAME")
Frac.GetXaxis().SetMoreLogLabels()
Frac.GetXaxis().SetNoExponent()
Frac_0.Draw("SAME")
if donu and Trgf<2:
    NFrac = hpts[Trgf].Clone("nfhpt_"+triggers[Trgf])
    NFrac_0 = hpt0s[Trgf].Clone("nfhpt0_"+triggers[Trgf])
    NFrac.Divide(nuhpt)
    NFrac_0.Divide(nuhpt0)
    NFrac.Scale(1.0/nuscale)
    NFrac_0.Scale(1.0/nuscale)
    NFrac.SetLineColor(kRed)
    NFrac_0.SetLineColor(kRed)
    NFrac.Draw("SAME")
    NFrac_0.Draw("SAME")
cuf.Draw()
cuf.SetLogx()
cuf.SetLogy()
cuf.Update()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cuf
